In [1]:
#r "nuget: Microsoft.ML"
using Microsoft.ML;
using System;
using System.Linq;

Installed package Microsoft.ML version 1.4.0

Create classes to map data to:

In [2]:
class SourceData
{
    public DateTime Date { get; set; }
}

class MappedData
{
    public DateTime SourceDate { get; set; }
    public int Month { get; set; }
    public bool IsWeekend { get; set; }

    public override string ToString() =>
        $"{SourceDate, 25} {Month, 5} {IsWeekend, 9}";
}

Instantiate a new MLContext and create some sample data:

In [3]:
var context = new MLContext();

var data = new[]
{
    new SourceData { Date = DateTime.Parse("2019-08-20") },
    new SourceData { Date = DateTime.Parse("2019-07-06") },
    new SourceData { Date = DateTime.Parse("2019-06-16") },
    new SourceData { Date = DateTime.Parse("2019-05-14") },
};
data

index,Date
0,2019-08-20 00:00:00Z
1,2019-07-06 00:00:00Z
2,2019-06-16 00:00:00Z
3,2019-05-14 00:00:00Z


Create a new DataView from the sample data:

In [4]:
var dataView = context.Data.LoadFromEnumerable<SourceData>(data);

Here is where we define a CustomMapping to map from SourceData to MappedData:

In [5]:
var encoder = context.Transforms.CustomMapping<SourceData, MappedData>((input, output) =>
{
    output.SourceDate = input.Date;
    output.Month = input.Date.Month;
    output.IsWeekend = input.Date.DayOfWeek == DayOfWeek.Saturday || input.Date.DayOfWeek == DayOfWeek.Sunday;
}, "MapDate");

Fit the encoder to our data:

In [6]:
var transformer = encoder.Fit(dataView);

Transform our data using the fitted transformer:

In [7]:
var transformedDataView = transformer.Transform(dataView);

Print our transformed data:

In [8]:
context.Data.CreateEnumerable<MappedData>(transformedDataView, reuseRowObject: false)

index,SourceDate,Month,IsWeekend
0,2019-08-20 00:00:00Z,8,False
1,2019-07-06 00:00:00Z,7,True
2,2019-06-16 00:00:00Z,6,True
3,2019-05-14 00:00:00Z,5,False
